In [1]:
import datetime, logging, os, sys, math, random
logging.disable(logging.WARNING)
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "3"

import tensorflow as tf
import tensorflow_addons as tfa
from keras import Sequential, layers
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import cv2 as cv
from PIL import Image as Img
import numba


import pickle 
import albumentations as A

from ImageDataAugmentor.image_data_augmentor import *
from albumentations.core.composition import Compose, OneOf
from albumentations.core.transforms_interface import ImageOnlyTransform

devices = tf.config.list_physical_devices('GPU')
for device in devices:
   tf.config.experimental.set_memory_growth(device, True) 
print(devices)

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU'), PhysicalDevice(name='/physical_device:GPU:1', device_type='GPU')]


In [2]:
PATH = os.path.abspath(os.path.join(os.getcwd() ,"../../../../datasets/sorghum-id-fgvc-9"))+"/"

MODEL_FOLDER = "sorghum-id-2022-04-30-124425-EfficientNetV2L-imagenet" #### CHANGE ME
MODEL_NAME   = "EfficientNetV2L-optimal.h5"
MODEL_PATH = "../models/"+MODEL_FOLDER+"/"+MODEL_NAME

test_dir = PATH+'test/'
save_dir    = '../results/sorghum/tta/'

print(MODEL_PATH)

../models/sorghum-id-2022-04-30-124425-EfficientNetV2L-imagenet/EfficientNetV2L-optimal.h5


In [3]:
f = open("../labels.pkl","rb")
labels = pickle.load(f)
f.close()

In [4]:
WIDTH = 512
HEIGHT = 512

submission = pd.read_csv(PATH+'sample_submission.csv')

submission

,filename,cultivar
0,1000005362.png,PI_152923
1,1000099707.png,PI_152923
2,1000135300.png,PI_152923
3,1000136796.png,PI_152923
4,1000292439.png,PI_152923
...,...,...
23634,999578153.png,PI_152923
23635,999692877.png,PI_152923
23636,999756998.png,PI_152923
23637,999892248.png,PI_152923


In [6]:
def clahe(img):
    c = cv.createCLAHE(clipLimit=40, tileGridSize=(16,16))  # create a clahe object
    t = np.asarray(img)                                     # convert to np array
    t = cv.cvtColor(t, cv.COLOR_BGR2HSV)                    # convert to OpenCV HSV
    t[:,:,-1] = c.apply(t[:,:,-1])                          # Apply CLAHE to the Value (greyscale) of the image
    t = cv.cvtColor(t, cv.COLOR_HSV2BGR)                    # Return to BGR OpenCV doamin
    t = Img.fromarray(t)                                    # Convert to PIL Image
    t = np.array(t)                                         # back to np array
    return t

def normalise(img):
    t = np.array(img,dtype=np.float32)/255
    return t

class CLAHE(ImageOnlyTransform):
    def apply(self, img, **params):
        return clahe(img)

class NORMALISE(ImageOnlyTransform):
    def apply(self, img, **params):
        return normalise(img)

def augment_data(phase: str):
    if phase == "train":
        return Compose([
                CLAHE(p=1),
                A.RandomResizedCrop(height=HEIGHT, width=WIDTH),
                A.Flip(p=0.5),
                A.RandomRotate90(p=0.5),
                A.ShiftScaleRotate(p=0.5),
                A.HueSaturationValue(p=0.5),
                A.OneOf([
                    A.RandomBrightnessContrast(p=0.5),
                    A.RandomGamma(p=0.5),
                ], p=0.5),
                OneOf([
                    A.Blur(p=0.1),
                    A.GaussianBlur(p=0.1),
                    A.MotionBlur(p=0.1),
                ], p=0.1),
                OneOf([
                    A.GaussNoise(p=0.1),
                    A.ISONoise(p=0.1),
                    A.GridDropout(ratio=0.5, p=0.2),
                    A.CoarseDropout(max_holes=16, min_holes=8, max_height=16,
                                    max_width=16, min_height=8, min_width=8, p=0.2)
                ], p=0.2),
                #A.Normalize(mean=[0.485, 0.456, 0.406],std=[0.229, 0.224, 0.225], max_pixel_value=255.0),
                
        ])
    else:
        return Compose([
                CLAHE(p=1),
                A.Resize(height=HEIGHT, width=WIDTH),
                #A.Normalize(mean=[0.485, 0.456, 0.406],std=[0.229, 0.224, 0.225], max_pixel_value=255.0),
        ])

## Test Time Augmentation - Horizontal Only

In [ ]:
TTA = [
        Compose([CLAHE(p=1),
                A.Resize(height=HEIGHT, width=WIDTH),]),

        Compose([CLAHE(p=1),
                A.Resize(height=HEIGHT, width=WIDTH),
                A.HorizontalFlip(p=1),]),

        # Compose([CLAHE(p=1),
        #         A.Resize(height=HEIGHT, width=WIDTH),
        #         A.VerticalFlip(p=1),]),
]

In [ ]:
image = cv2.imread("/mnt/shared/scratch/fmacfarl/datasets/sorghum-id-fgvc-9/train_images/2017-06-05__13-04-59-234.png")
#image = cv2.imread("/mnt/shared/scratch/fmacfarl/datasets/sorghum-id-fgvc-9/train_images/2017-06-30__12-38-34-583.png")
image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
for aug in TTA:
    transformed_image = aug(image=image)["image"]

    plt.figure(figsize=(10, 10))
    ax = plt.subplot(1, 2, 1)
    plt.imshow(image)
    ax = plt.subplot(1, 2, 2)
    plt.imshow(transformed_image)
    plt.show()

## Without TTA

In [ ]:
test_gen = ImageDataAugmentor(augment=augment_data("test"))
test_generator = test_gen.flow_from_dataframe(dataframe=submission,
                                              directory=test_dir,
                                              x_col='filename',
                                              y_col=None,
                                              target_size=(WIDTH,HEIGHT),
                                              color_mode='rgb',
                                              class_mode=None,
                                              batch_size=1,
                                              shuffle=False,)

STEP_SIZE_TEST=test_generator.n//test_generator.batch_size
STEP_SIZE_TEST,test_generator.n,test_generator.batch_size

reconstructed_model = tf.keras.models.load_model(MODEL_PATH)

%time
test_generator.reset()
results = reconstructed_model.predict(test_generator,verbose=1,steps=STEP_SIZE_TEST)

predicted_class_indices=np.argmax(results,axis=1)
labels_dict = dict((v,k) for k,v in labels.items())
predictions = [labels_dict[k] for k in predicted_class_indices]

filenames=test_generator.filenames
submission=pd.DataFrame({"Filename":[filename.replace('all_classes/','')for filename in filenames],
                      "cultivar":predictions})

submission_name = save_dir+'submission-no-TTA-EfficientNetV2L-imagenet-2022-04-30-124425.csv'
submission.to_csv(submission_name,index=False)

## With TTA and Average Confidence Aggregation

### Horizontal Flip 

In [ ]:
reconstructed_model = tf.keras.models.load_model(MODEL_PATH)

results_aggregated = np.zeros([23639,100],dtype=np.float32)
results_collated = np.zeros([23639,100,len(TTA)],dtype=np.float32)

submission = pd.read_csv(PATH+'sample_submission.csv')

tta_df = pd.DataFrame(index=submission.index)

for i, tta in enumerate(TTA):
    test_gen = ImageDataAugmentor(augment=tta)
    test_generator = test_gen.flow_from_dataframe(dataframe=submission,
                                              directory=test_dir,
                                              x_col='filename',
                                              y_col=None,
                                              target_size=(WIDTH,HEIGHT),
                                              class_mode=None,
                                              batch_size=1,
                                              shuffle=False,)

    STEP_SIZE_TEST=test_generator.n//test_generator.batch_size
    STEP_SIZE_TEST,test_generator.n,test_generator.batch_size
    
    test_generator.reset()
    # x= test_generator.next()
    # image = x[0]
    # plt.imshow(image/255)
    # plt.show()  
    # print(np.max(image),np.min(image))
    results = reconstructed_model.predict(test_generator,verbose=1,steps=STEP_SIZE_TEST)
    results_collated[:,:,i] = results
    results_aggregated += results


results_aggregated = results_aggregated/len(TTA)
predicted_class_indices=np.argmax(results_aggregated,axis=1)

labels_dict = dict((v,k) for k,v in labels.items())
predictions = [labels_dict[k] for k in predicted_class_indices]

filenames=test_generator.filenames
submission=pd.DataFrame({"Filename":[filename.replace('all_classes/','')for filename in filenames],
                      "cultivar":predictions})

submission_name = save_dir+'submission-TTA-hf-EfficientNetV2L-imagenet-2022-04-30-124425.csv'
submission.to_csv(submission_name,index=False)



### Vertical Flip

In [ ]:
TTA = [
        Compose([CLAHE(p=1),
                A.Resize(height=HEIGHT, width=WIDTH),]),

        # Compose([CLAHE(p=1),
        #         A.Resize(height=HEIGHT, width=WIDTH),
        #         A.HorizontalFlip(p=1),]),

        Compose([CLAHE(p=1),
                A.Resize(height=HEIGHT, width=WIDTH),
                A.VerticalFlip(p=1),]),
]

In [ ]:
reconstructed_model = tf.keras.models.load_model(MODEL_PATH)

results_aggregated = np.zeros([23639,100],dtype=np.float32)
results_collated = np.zeros([23639,100,len(TTA)],dtype=np.float32)

submission = pd.read_csv(PATH+'sample_submission.csv')

tta_df = pd.DataFrame(index=submission.index)

for i, tta in enumerate(TTA):
    test_gen = ImageDataAugmentor(augment=tta)
    test_generator = test_gen.flow_from_dataframe(dataframe=submission,
                                              directory=test_dir,
                                              x_col='filename',
                                              y_col=None,
                                              target_size=(WIDTH,HEIGHT),
                                              class_mode=None,
                                              batch_size=1,
                                              shuffle=False,)

    STEP_SIZE_TEST=test_generator.n//test_generator.batch_size
    STEP_SIZE_TEST,test_generator.n,test_generator.batch_size
    
    test_generator.reset()
    # x= test_generator.next()
    # image = x[0]
    # plt.imshow(image/255)
    # plt.show()  
    # print(np.max(image),np.min(image))
    results = reconstructed_model.predict(test_generator,verbose=1,steps=STEP_SIZE_TEST)
    results_collated[:,:,i] = results
    results_aggregated += results


results_aggregated = results_aggregated/len(TTA)
predicted_class_indices=np.argmax(results_aggregated,axis=1)

labels_dict = dict((v,k) for k,v in labels.items())
predictions = [labels_dict[k] for k in predicted_class_indices]

filenames=test_generator.filenames
submission=pd.DataFrame({"Filename":[filename.replace('all_classes/','')for filename in filenames],
                      "cultivar":predictions})

submission_name = save_dir+'submission-TTA-vf-EfficientNetV2L-imagenet-2022-04-30-124425.csv'
submission.to_csv(submission_name,index=False)


### Horizontal + Vertical (Individual Flips)

In [ ]:
TTA = [
        Compose([CLAHE(p=1),
                A.Resize(height=HEIGHT, width=WIDTH),]),

        Compose([CLAHE(p=1),
                A.Resize(height=HEIGHT, width=WIDTH),
                A.HorizontalFlip(p=1),]),

        Compose([CLAHE(p=1),
                A.Resize(height=HEIGHT, width=WIDTH),
                A.VerticalFlip(p=1),]),
]

In [ ]:
reconstructed_model = tf.keras.models.load_model(MODEL_PATH)

results_aggregated = np.zeros([23639,100],dtype=np.float32)
results_collated = np.zeros([23639,100,len(TTA)],dtype=np.float32)

submission = pd.read_csv(PATH+'sample_submission.csv')

tta_df = pd.DataFrame(index=submission.index)

for i, tta in enumerate(TTA):
    test_gen = ImageDataAugmentor(augment=tta)
    test_generator = test_gen.flow_from_dataframe(dataframe=submission,
                                              directory=test_dir,
                                              x_col='filename',
                                              y_col=None,
                                              target_size=(WIDTH,HEIGHT),
                                              class_mode=None,
                                              batch_size=1,
                                              shuffle=False,)

    STEP_SIZE_TEST=test_generator.n//test_generator.batch_size
    STEP_SIZE_TEST,test_generator.n,test_generator.batch_size
    
    test_generator.reset()
    # x= test_generator.next()
    # image = x[0]
    # plt.imshow(image/255)
    # plt.show()  
    # print(np.max(image),np.min(image))
    results = reconstructed_model.predict(test_generator,verbose=1,steps=STEP_SIZE_TEST)
    results_collated[:,:,i] = results
    results_aggregated += results


results_aggregated = results_aggregated/len(TTA)
predicted_class_indices=np.argmax(results_aggregated,axis=1)

labels_dict = dict((v,k) for k,v in labels.items())
predictions = [labels_dict[k] for k in predicted_class_indices]

filenames=test_generator.filenames
submission=pd.DataFrame({"Filename":[filename.replace('all_classes/','')for filename in filenames],
                      "cultivar":predictions})

submission_name = save_dir+'submission-TTA-hfvf-EfficientNetV2L-imagenet-2022-04-30-124425.csv'
submission.to_csv(submission_name,index=False)

### Combinations of Horizontal + Vertical Flips through rotations

In [ ]:
TTA = [
        Compose([CLAHE(p=1),
                A.Resize(height=HEIGHT, width=WIDTH),
                A.Affine(rotate=0,p=1),]),

        # Compose([CLAHE(p=1),
        #         A.Resize(height=HEIGHT, width=WIDTH),
        #         A.Affine(rotate=90,p=1),]),

        Compose([CLAHE(p=1),
                A.Resize(height=HEIGHT, width=WIDTH),
                A.Affine(rotate=180,p=2),]),
        
        # Compose([CLAHE(p=1),
        #         A.Resize(height=HEIGHT, width=WIDTH),
        #         A.Affine(rotate=270,p=3),]),

        Compose([CLAHE(p=1),
                A.Resize(height=HEIGHT, width=WIDTH),
                A.HorizontalFlip(p=1),
                A.Affine(rotate=0,p=1),]),

        # Compose([CLAHE(p=1),
        #         A.Resize(height=HEIGHT, width=WIDTH),
        #         A.HorizontalFlip(p=1),
        #         A.Affine(rotate=90,p=1),]),

        Compose([CLAHE(p=1),
                A.Resize(height=HEIGHT, width=WIDTH),
                A.HorizontalFlip(p=1),
                A.Affine(rotate=180,p=2),]),
        
        # Compose([CLAHE(p=1),
        #         A.Resize(height=HEIGHT, width=WIDTH),
        #         A.HorizontalFlip(p=1),
        #         A.Affine(rotate=270,p=3),]),

        # Compose([CLAHE(p=1),
        #         A.Resize(height=HEIGHT, width=WIDTH),
        #         A.ShiftScaleRotate(p=1),]),
]

In [ ]:
reconstructed_model = tf.keras.models.load_model(MODEL_PATH)

results_aggregated = np.zeros([23639,100],dtype=np.float32)
results_collated = np.zeros([23639,100,len(TTA)],dtype=np.float32)

submission = pd.read_csv(PATH+'sample_submission.csv')

tta_df = pd.DataFrame(index=submission.index)

for i, tta in enumerate(TTA):
    test_gen = ImageDataAugmentor(augment=tta)
    test_generator = test_gen.flow_from_dataframe(dataframe=submission,
                                              directory=test_dir,
                                              x_col='filename',
                                              y_col=None,
                                              target_size=(WIDTH,HEIGHT),
                                              class_mode=None,
                                              batch_size=1,
                                              shuffle=False,)

    STEP_SIZE_TEST=test_generator.n//test_generator.batch_size
    STEP_SIZE_TEST,test_generator.n,test_generator.batch_size
    
    test_generator.reset()
    # x= test_generator.next()
    # image = x[0]
    # plt.imshow(image/255)
    # plt.show()  
    # print(np.max(image),np.min(image))
    results = reconstructed_model.predict(test_generator,verbose=1,steps=STEP_SIZE_TEST)
    results_collated[:,:,i] = results
    results_aggregated += results


results_aggregated = results_aggregated/len(TTA)
predicted_class_indices=np.argmax(results_aggregated,axis=1)

labels_dict = dict((v,k) for k,v in labels.items())
predictions = [labels_dict[k] for k in predicted_class_indices]

filenames=test_generator.filenames
submission=pd.DataFrame({"Filename":[filename.replace('all_classes/','')for filename in filenames],
                      "cultivar":predictions})

submission_name = save_dir+'submission-TTA-hf+vf-EfficientNetV2L-imagenet-2022-04-30-124425.csv'
submission.to_csv(submission_name,index=False)

### 90 Degree Rotations

In [ ]:
TTA = [
        Compose([CLAHE(p=1),
                A.Resize(height=HEIGHT, width=WIDTH),
                A.Affine(rotate=0,p=1),]),

        Compose([CLAHE(p=1),
                A.Resize(height=HEIGHT, width=WIDTH),
                A.Affine(rotate=90,p=1),]),

        Compose([CLAHE(p=1),
                A.Resize(height=HEIGHT, width=WIDTH),
                A.Affine(rotate=180,p=2),]),
        
        Compose([CLAHE(p=1),
                A.Resize(height=HEIGHT, width=WIDTH),
                A.Affine(rotate=270,p=3),]),

        Compose([CLAHE(p=1),
                A.Resize(height=HEIGHT, width=WIDTH),
                A.HorizontalFlip(p=1),
                A.Affine(rotate=0,p=1),]),

        Compose([CLAHE(p=1),
                A.Resize(height=HEIGHT, width=WIDTH),
                A.HorizontalFlip(p=1),
                A.Affine(rotate=90,p=1),]),

        Compose([CLAHE(p=1),
                A.Resize(height=HEIGHT, width=WIDTH),
                A.HorizontalFlip(p=1),
                A.Affine(rotate=180,p=2),]),
        
        Compose([CLAHE(p=1),
                A.Resize(height=HEIGHT, width=WIDTH),
                A.HorizontalFlip(p=1),
                A.Affine(rotate=270,p=3),]),

        # Compose([CLAHE(p=1),
        #         A.Resize(height=HEIGHT, width=WIDTH),
        #         A.ShiftScaleRotate(p=1),]),
]

In [ ]:
reconstructed_model = tf.keras.models.load_model(MODEL_PATH)

results_aggregated = np.zeros([23639,100],dtype=np.float32)
results_collated = np.zeros([23639,100,len(TTA)],dtype=np.float32)

submission = pd.read_csv(PATH+'sample_submission.csv')

tta_df = pd.DataFrame(index=submission.index)

for i, tta in enumerate(TTA):
    test_gen = ImageDataAugmentor(augment=tta)
    test_generator = test_gen.flow_from_dataframe(dataframe=submission,
                                              directory=test_dir,
                                              x_col='filename',
                                              y_col=None,
                                              target_size=(WIDTH,HEIGHT),
                                              class_mode=None,
                                              batch_size=1,
                                              shuffle=False,)

    STEP_SIZE_TEST=test_generator.n//test_generator.batch_size
    STEP_SIZE_TEST,test_generator.n,test_generator.batch_size
    
    test_generator.reset()
    # x= test_generator.next()
    # image = x[0]
    # plt.imshow(image/255)
    # plt.show()  
    # print(np.max(image),np.min(image))
    results = reconstructed_model.predict(test_generator,verbose=1,steps=STEP_SIZE_TEST)
    results_collated[:,:,i] = results
    results_aggregated += results


results_aggregated = results_aggregated/len(TTA)
predicted_class_indices=np.argmax(results_aggregated,axis=1)

labels_dict = dict((v,k) for k,v in labels.items())
predictions = [labels_dict[k] for k in predicted_class_indices]

filenames=test_generator.filenames
submission=pd.DataFrame({"Filename":[filename.replace('all_classes/','')for filename in filenames],
                      "cultivar":predictions})

submission_name = save_dir+'submission-TTA-rot-EfficientNetV2L-imagenet-2022-04-30-124425.csv'
submission.to_csv(submission_name,index=False)

### Horizontal Flip, Vertical Flip and a Random Rotation

In [6]:
TTA = [
        Compose([CLAHE(p=1),
                A.Resize(height=HEIGHT, width=WIDTH),]),

        Compose([CLAHE(p=1),
                A.Resize(height=HEIGHT, width=WIDTH),
                A.HorizontalFlip(p=1),]),

        Compose([CLAHE(p=1),
                A.Resize(height=HEIGHT, width=WIDTH),
                A.VerticalFlip(p=1),]),

        Compose([CLAHE(p=1),
                A.Resize(height=HEIGHT, width=WIDTH),
                A.ShiftScaleRotate(p=1),]),
]

In [7]:
reconstructed_model = tf.keras.models.load_model(MODEL_PATH)

results_aggregated = np.zeros([23639,100],dtype=np.float32)
results_collated = np.zeros([23639,100,len(TTA)],dtype=np.float32)

submission = pd.read_csv(PATH+'sample_submission.csv')

tta_df = pd.DataFrame(index=submission.index)

for i, tta in enumerate(TTA):
    test_gen = ImageDataAugmentor(augment=tta)
    test_generator = test_gen.flow_from_dataframe(dataframe=submission,
                                              directory=test_dir,
                                              x_col='filename',
                                              y_col=None,
                                              target_size=(WIDTH,HEIGHT),
                                              class_mode=None,
                                              batch_size=1,
                                              shuffle=False,)

    STEP_SIZE_TEST=test_generator.n//test_generator.batch_size
    STEP_SIZE_TEST,test_generator.n,test_generator.batch_size
    
    test_generator.reset()
    # x= test_generator.next()
    # image = x[0]
    # plt.imshow(image/255)
    # plt.show()  
    # print(np.max(image),np.min(image))
    results = reconstructed_model.predict(test_generator,verbose=1,steps=STEP_SIZE_TEST)
    results_collated[:,:,i] = results
    results_aggregated += results


results_aggregated = results_aggregated/len(TTA)
predicted_class_indices=np.argmax(results_aggregated,axis=1)

labels_dict = dict((v,k) for k,v in labels.items())
predictions = [labels_dict[k] for k in predicted_class_indices]

filenames=test_generator.filenames
submission=pd.DataFrame({"Filename":[filename.replace('all_classes/','')for filename in filenames],
                      "cultivar":predictions})

submission_name = save_dir+'submission-TTA-hfvfrot-EfficientNetV2L-imagenet-2022-04-30-124425.csv'
submission.to_csv(submission_name,index=False)

Found 23639 validated image filenames.
23639/23639 [==============================] - 1246s 52ms/step
Found 23639 validated image filenames.
23639/23639 [==============================] - 1260s 53ms/step
Found 23639 validated image filenames.
23639/23639 [==============================] - 1095s 46ms/step
Found 23639 validated image filenames.
23639/23639 [==============================] - 1096s 46ms/step


### 90 Degree Rotations and a Ramndom Angle Rotation 

This may make our predictions worse as camera is mounted on linear translation gantry

In [8]:
TTA = [
        Compose([CLAHE(p=1),
                A.Resize(height=HEIGHT, width=WIDTH),
                A.Affine(rotate=0,p=1),]),

        Compose([CLAHE(p=1),
                A.Resize(height=HEIGHT, width=WIDTH),
                A.Affine(rotate=90,p=1),]),

        Compose([CLAHE(p=1),
                A.Resize(height=HEIGHT, width=WIDTH),
                A.Affine(rotate=180,p=2),]),
        
        Compose([CLAHE(p=1),
                A.Resize(height=HEIGHT, width=WIDTH),
                A.Affine(rotate=270,p=3),]),

        Compose([CLAHE(p=1),
                A.Resize(height=HEIGHT, width=WIDTH),
                A.HorizontalFlip(p=1),
                A.Affine(rotate=0,p=1),]),

        Compose([CLAHE(p=1),
                A.Resize(height=HEIGHT, width=WIDTH),
                A.HorizontalFlip(p=1),
                A.Affine(rotate=90,p=1),]),

        Compose([CLAHE(p=1),
                A.Resize(height=HEIGHT, width=WIDTH),
                A.HorizontalFlip(p=1),
                A.Affine(rotate=180,p=2),]),
        
        Compose([CLAHE(p=1),
                A.Resize(height=HEIGHT, width=WIDTH),
                A.HorizontalFlip(p=1),
                A.Affine(rotate=270,p=3),]),

        Compose([CLAHE(p=1),
                A.Resize(height=HEIGHT, width=WIDTH),
                A.ShiftScaleRotate(p=1),]),
]

In [9]:
reconstructed_model = tf.keras.models.load_model(MODEL_PATH)

results_aggregated = np.zeros([23639,100],dtype=np.float32)
results_collated = np.zeros([23639,100,len(TTA)],dtype=np.float32)

submission = pd.read_csv(PATH+'sample_submission.csv')

tta_df = pd.DataFrame(index=submission.index)

for i, tta in enumerate(TTA):
    test_gen = ImageDataAugmentor(augment=tta)
    test_generator = test_gen.flow_from_dataframe(dataframe=submission,
                                              directory=test_dir,
                                              x_col='filename',
                                              y_col=None,
                                              target_size=(WIDTH,HEIGHT),
                                              class_mode=None,
                                              batch_size=1,
                                              shuffle=False,)

    STEP_SIZE_TEST=test_generator.n//test_generator.batch_size
    STEP_SIZE_TEST,test_generator.n,test_generator.batch_size
    
    test_generator.reset()
    # x= test_generator.next()
    # image = x[0]
    # plt.imshow(image/255)
    # plt.show()  
    # print(np.max(image),np.min(image))
    results = reconstructed_model.predict(test_generator,verbose=1,steps=STEP_SIZE_TEST)
    results_collated[:,:,i] = results
    results_aggregated += results


results_aggregated = results_aggregated/len(TTA)
predicted_class_indices=np.argmax(results_aggregated,axis=1)

labels_dict = dict((v,k) for k,v in labels.items())
predictions = [labels_dict[k] for k in predicted_class_indices]

filenames=test_generator.filenames
submission=pd.DataFrame({"Filename":[filename.replace('all_classes/','')for filename in filenames],
                      "cultivar":predictions})

submission_name = save_dir+'submission-TTA-rot+rnd-EfficientNetV2L-imagenet-2022-04-30-124425.csv'
submission.to_csv(submission_name,index=False)

Found 23639 validated image filenames.
23639/23639 [==============================] - 1105s 47ms/step
Found 23639 validated image filenames.
23639/23639 [==============================] - 1110s 47ms/step
Found 23639 validated image filenames.
23639/23639 [==============================] - 1097s 46ms/step
Found 23639 validated image filenames.
23639/23639 [==============================] - 1098s 46ms/step
Found 23639 validated image filenames.
23639/23639 [==============================] - 1088s 46ms/step
Found 23639 validated image filenames.
23639/23639 [==============================] - 1095s 46ms/step
Found 23639 validated image filenames.
23639/23639 [==============================] - 1097s 46ms/step
Found 23639 validated image filenames.
23639/23639 [==============================] - 1095s 46ms/step
Found 23639 validated image filenames.
23639/23639 [==============================] - 1103s 47ms/step


### Horizontal and Vertical Flips, Random Rotation and Colour Jitter

In [7]:
TTA = [
        Compose([CLAHE(p=1),
                A.Resize(height=HEIGHT, width=WIDTH),]),

        Compose([CLAHE(p=1),
                A.Resize(height=HEIGHT, width=WIDTH),
                A.HorizontalFlip(p=1),]),

        Compose([CLAHE(p=1),
                A.Resize(height=HEIGHT, width=WIDTH),
                A.VerticalFlip(p=1),]),

        Compose([CLAHE(p=1),
                A.Resize(height=HEIGHT, width=WIDTH),
                A.ShiftScaleRotate(p=1),]),
        
        Compose([CLAHE(p=1),
                A.Resize(height=HEIGHT, width=WIDTH),
                A.ColorJitter (brightness=0.2, contrast=0.2, saturation=0.2, hue=0.2, p=1),]),
]

In [10]:
reconstructed_model = tf.keras.models.load_model(MODEL_PATH)

results_aggregated = np.zeros([23639,100],dtype=np.float32)
results_collated = np.zeros([23639,100,len(TTA)],dtype=np.float32)

submission = pd.read_csv(PATH+'sample_submission.csv')

tta_df = pd.DataFrame(index=submission.index)

for i, tta in enumerate(TTA):
    test_gen = ImageDataAugmentor(augment=tta)
    test_generator = test_gen.flow_from_dataframe(dataframe=submission,
                                              directory=test_dir,
                                              x_col='filename',
                                              y_col=None,
                                              target_size=(WIDTH,HEIGHT),
                                              class_mode=None,
                                              batch_size=1,
                                              shuffle=False,)

    STEP_SIZE_TEST=test_generator.n//test_generator.batch_size
    STEP_SIZE_TEST,test_generator.n,test_generator.batch_size
    
    # test_generator.reset()
    # x= test_generator.next()
    # image = x[0]
    # plt.imshow(image/255)
    # plt.show()  
    # print(np.max(image),np.min(image))
    results = reconstructed_model.predict(test_generator,verbose=1,steps=STEP_SIZE_TEST)
    results_collated[:,:,i] = results
    results_aggregated += results


results_aggregated = results_aggregated/len(TTA)
predicted_class_indices=np.argmax(results_aggregated,axis=1)

labels_dict = dict((v,k) for k,v in labels.items())
predictions = [labels_dict[k] for k in predicted_class_indices]

filenames=test_generator.filenames
submission=pd.DataFrame({"Filename":[filename.replace('all_classes/','')for filename in filenames],
                      "cultivar":predictions})

submission_name = save_dir+'submission-TTA-hfvfrndrot+jitter-EfficientNetV2L-imagenet-2022-04-30-124425.csv'
submission.to_csv(submission_name,index=False)

Found 23639 validated image filenames.
23639/23639 [==============================] - 1549s 65ms/step
